In [26]:
!pip install ace_tools

In [27]:
import pandas as pd

# Load the merged CSV
df = pd.read_csv('/content/NBA-Cleaned-Merged.csv')

# Preview
df.head()


,EVENTID,EVENTNUM,GAME_ID,HOMEDESCRIPTION,PCTIMESTRING,PERIOD,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PLAYER1_TEAM_CITY,...,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,VISITORDESCRIPTION,SCORE,SCOREMARGIN,HOME_SCORE,VISITOR_SCORE
0,2,2,20001116,NaN,2025-04-19T11:45:00.000Z,1,947,Allen Iverson,PHI,Philadelphia,...,NaN,NaN,NaN,NaN,NaN,Iverson 21' Jump Shot (2 PTS),2 - 0,-2,2,0
1,9,9,20001116,NaN,2025-04-19T10:44:00.000Z,1,248,George Lynch,PHI,Philadelphia,...,NaN,NaN,NaN,NaN,NaN,Lynch Layup (2 PTS) (Snow 1 AST),04-Feb,-2,4,2
2,19,19,20001116,NaN,2025-04-19T09:45:00.000Z,1,947,Allen Iverson,PHI,Philadelphia,...,NaN,NaN,NaN,NaN,NaN,Iverson Driving Layup (4 PTS),06-Feb,-4,6,2
3,23,24,20001116,NaN,2025-04-19T09:15:00.000Z,1,947,Allen Iverson,PHI,Philadelphia,...,NaN,NaN,NaN,NaN,NaN,Iverson 24' 3PT Jump Shot (7 PTS) (Mutombo 1 AST),09-Feb,-7,9,2
4,24,25,20001116,Grant 17' Jump Shot (2 PTS) (Majerle 1 AST),2025-04-19T08:54:00.000Z,1,258,Brian Grant,MIA,Miami,...,NaN,NaN,NaN,NaN,NaN,NaN,09-Apr,-5,9,4


Most Scoring Quarters

In [28]:
# Ensure score columns are integers
df['HOME_SCORE'] = pd.to_numeric(df['HOME_SCORE'], errors='coerce')
df['VISITOR_SCORE'] = pd.to_numeric(df['VISITOR_SCORE'], errors='coerce')

# Drop rows where scores are missing
df = df.dropna(subset=['HOME_SCORE', 'VISITOR_SCORE'])


In [29]:
# Group home team scores
home_scores = df.groupby(['PLAYER1_TEAM_NICKNAME', 'PERIOD'])['HOME_SCORE'].max().reset_index()
home_scores.columns = ['Team', 'Quarter', 'Score']

# Group visitor team scores
visitor_scores = df.groupby(['PLAYER2_TEAM_NICKNAME', 'PERIOD'])['VISITOR_SCORE'].max().reset_index()
visitor_scores.columns = ['Team', 'Quarter', 'Score']

# Combine both
all_scores = pd.concat([home_scores, visitor_scores])

# Drop rows with no team name
all_scores = all_scores.dropna(subset=['Team'])

# Group again to get total scores per team per quarter
team_quarter_scores = all_scores.groupby(['Team', 'Quarter'])['Score'].sum().reset_index()

# For each team, find the quarter with the highest total score
most_scoring_quarters = team_quarter_scores.loc[
    team_quarter_scores.groupby('Team')['Score'].idxmax()
].reset_index(drop=True)

# Sort alphabetically
most_scoring_quarters = most_scoring_quarters.sort_values(by='Team')


In [30]:
most_scoring_quarters.head()

,Team,Quarter,Score
0,76ers,4,213
1,Bucks,4,233
2,Bulls,4,201
3,Cavaliers,4,219
4,Celtics,4,205


In [31]:
most_scoring_quarters.to_csv("most_scoring_quarters.csv")

Top 5 Teams

In [32]:
# Group total home scores per team
home_team_scores = df.groupby('PLAYER1_TEAM_NICKNAME')['HOME_SCORE'].max().reset_index()
home_team_scores.columns = ['Team', 'Points_Home']

# Group total visitor scores per team
visitor_team_scores = df.groupby('PLAYER2_TEAM_NICKNAME')['VISITOR_SCORE'].max().reset_index()
visitor_team_scores.columns = ['Team', 'Points_Away']

# Combine both with outer join to handle teams appearing in only one role
total_scores = pd.merge(home_team_scores, visitor_team_scores, on='Team', how='outer').fillna(0)

# Sum both home and away scores
total_scores['Total_Points'] = total_scores['Points_Home'] + total_scores['Points_Away']

# Get top 5 scoring teams
top_5_teams = total_scores.sort_values(by='Total_Points', ascending=False).head(5).reset_index(drop=True)



In [33]:
top_5_teams.head()

,Team,Points_Home,Points_Away,Total_Points
0,Hawks,127,124,251
1,Pistons,129,115,244
2,Nets,120,124,244
3,Lakers,119,124,243
4,Pacers,118,121,239


In [34]:
top_5_teams.to_csv("top_5_teams.csv")

Percentage of players who have scored 40 points or more in a single match


In [35]:
import pandas as pd
import re

# Function to extract points from scoring descriptions
def extract_points(description):
    if pd.isna(description):
        return 0
    match = re.search(r'\((\d+)\s+PTS\)', description)
    return int(match.group(1)) if match else 0

# Apply the point extraction function
df['points_scored'] = df.apply(
    lambda row: extract_points(row['HOMEDESCRIPTION']) if pd.notna(row['HOMEDESCRIPTION']) else extract_points(row['VISITORDESCRIPTION']),
    axis=1
)

# Aggregate points per player per game
player_game_points = df.groupby(['PLAYER1_NAME', 'EVENTNUM'])['points_scored'].sum().reset_index()
player_game_points.columns = ['Player', 'Game_ID', 'Total_Points']

# Filter those who scored 40+
players_40_or_more = player_game_points[player_game_points['Total_Points'] >= 40].sort_values(by='Total_Points', ascending=False).reset_index(drop=True)

# Calculate percentage
total_players = len(player_game_points)
total_40_plus = len(players_40_or_more)
percentage_40_plus = (total_40_plus / total_players) * 100

print(f"{total_40_plus} out of {total_players} players scored 40+ points in a single match.")
print(f"Percentage: {percentage_40_plus:.2f}%")

54 out of 8802 players scored 40+ points in a single match.
Percentage: 0.61%


In [36]:
players_40_or_more.head()

,Player,Game_ID,Total_Points
0,Jerry Stackhouse,421,71
1,Kobe Bryant,428,65
2,Shaquille O'Neal,451,65
3,Allen Iverson,358,63
4,Antonio McDyess,415,62


In [41]:
players_40_or_more.to_csv("players_40_or_more.csv")

The total number of matches won, lost

In [38]:
# Step 1: Get final scores per match
match_scores = df.groupby('EVENTNUM').agg({
    'PLAYER1_TEAM_NICKNAME': 'last',
    'PLAYER2_TEAM_NICKNAME': 'last',
    'HOME_SCORE': 'max',
    'VISITOR_SCORE': 'max'
}).reset_index()

match_scores.columns = ['Match_ID', 'Home_Team', 'Away_Team', 'Home_Score', 'Visitor_Score']

# Step 2: Determine results
results = []

for _, row in match_scores.iterrows():
    if row['Home_Score'] > row['Visitor_Score']:
        results.append({'Team': row['Home_Team'], 'Result': 'Win'})
        results.append({'Team': row['Away_Team'], 'Result': 'Loss'})
    elif row['Visitor_Score'] > row['Home_Score']:
        results.append({'Team': row['Away_Team'], 'Result': 'Win'})
        results.append({'Team': row['Home_Team'], 'Result': 'Loss'})
    else:
        # In case of draw (rare)
        results.append({'Team': row['Home_Team'], 'Result': 'Draw'})
        results.append({'Team': row['Away_Team'], 'Result': 'Draw'})

results_df = pd.DataFrame(results)

# Step 3: Count wins/losses per team
team_results = results_df.groupby(['Team', 'Result']).size().unstack(fill_value=0).reset_index()

# Optional: Fill missing 'Win' or 'Loss' with 0
if 'Win' not in team_results.columns:
    team_results['Win'] = 0
if 'Loss' not in team_results.columns:
    team_results['Loss'] = 0

# Step 4: Identify top scoring player per team per match
player_scores = df.groupby(['EVENTNUM', 'PLAYER1_TEAM_NICKNAME', 'PLAYER1_NAME'])['points_scored'].sum().reset_index()
player_scores.columns = ['Match_ID', 'Team', 'Player', 'Total_Points']

# Get top scorer per team per match
top_scorers = player_scores.sort_values(['Match_ID', 'Team', 'Total_Points'], ascending=[True, True, False]).drop_duplicates(['Match_ID', 'Team'])

# Step 5: Join top scorers to win/loss results (optional summary)
summary_df = team_results.merge(
    top_scorers.groupby('Team')['Player'].agg(lambda x: x.mode()[0] if not x.mode().empty else None).reset_index(),
    on='Team', how='left'
)
summary_df.rename(columns={'Player': 'Top_Scorer'}, inplace=True)



In [39]:
summary_df.head()

,Team,Draw,Loss,Win,Top_Scorer
0,76ers,4,15,18,Allen Iverson
1,Bucks,3,30,33,Glenn Robinson
2,Bulls,7,22,29,Elton Brand
3,Cavaliers,0,7,13,Andre Miller
4,Celtics,0,3,4,Paul Pierce


In [40]:
summary_df.to_csv("summary_df.csv")